In [2]:
import csv
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon

In [6]:
data = pd.read_csv("../data/joined_raw_airports.csv")

In [19]:
map_df = data.drop(columns=["id","carrier_ct", "carrier", "carrier_name", "weather_ct", "nas_ct","security_ct","late_aircraft_ct", "arr_diverted", "arr_delay","carrier_delay", "weather_delay","nas_delay", "security_delay", "late_aircraft_delay", "id-2","country_code"])
map_df.head()

,year,month,airport_name,arr_flights,arr_del15,arr_cancelled,iata,airport,latitude,longitude
0,2020,12,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",5985,445,5,ATL,Hartsfield-Jackson Atlanta International Airport,33.6367,-84.4281
1,2020,12,"Boston, MA: Logan International",21,3,0,BOS,Logan International Airport,42.3643,-71.0052
2,2020,12,"Baltimore, MD: Baltimore/Washington Internatio...",96,17,1,BWI,Baltimore/Washington International Airport,39.1753,-76.6683
3,2020,12,"Charlotte, NC: Charlotte Douglas International",42,1,0,CLT,Charlotte Douglas International Airport,35.2140,-80.9431
4,2020,12,"Washington, DC: Ronald Reagan Washington National",37,1,1,DCA,Ronald Reagan Washington National Airport,38.8522,-77.0378


In [27]:
airports_table = pd.read_csv("airport_table.csv")
airports_table.reset_index(drop=True)
del airports_table["Unnamed: 0"]
display(airports_table)

,iata,airport,latitude,longitude
0,SLC,Salt Lake City International Airport,40.7884,-111.9780
1,ATL,Hartsfield-Jackson Atlanta International Airport,33.6367,-84.4281
2,FLL,Fort Lauderdale-Hollywood International Airport,26.0726,-80.1527
3,MIA,Miami International Airport,25.7932,-80.2906
4,MCO,Orlando International Airport,28.4294,-81.3090
5,MSP,Minneapolis-Saint Paul International Airport (...,44.8820,-93.2218
6,LAX,Los Angeles International Airport,33.9425,-118.4080
7,LAS,Harry Reid International Airport,36.0801,-115.1520
8,DFW,Dallas/Fort Worth International Airport,32.8968,-97.0380
9,DEN,Denver International Airport,39.8617,-104.6730


In [9]:
m_airports = folium.Map(location=(39.8283, -98.5795),tiles="Cartodb dark_matter", zoom_start=4)

In [15]:

#calculate percentages
delay_percents = round((map_df.groupby('iata')['arr_del15'].sum()/map_df.groupby('iata')["arr_flights"].sum())*100, 3)
cancel_percents = round((map_df.groupby('iata')['arr_cancelled'].sum()/map_df.groupby('iata')["arr_flights"].sum())*100, 3)

#gather latitude and longitue
airport_iata = map_df.groupby('iata')['airport'].unique()

In [16]:
# Convert the DataFrame columns to native Python lists and types
lat = map_df.groupby('iata')['latitude'].unique().astype(float).tolist()
lng = map_df.groupby('iata')['longitude'].unique().astype(float).tolist()

delay = delay_percents.astype(float).tolist()
cancel = cancel_percents.astype(float).tolist()

# Zip the lists for folium format
delay_heat = list(zip(lat, lng, delay))
cancel_heat = list(zip(lat, lng, cancel))

In [17]:
# Create map features
marker_cluster = MarkerCluster(name="Airports").add_to(m_airports)

for row in airports_table.itertuples():
    folium.Marker(
        location=[row.latitude, row.longitude],
        popup=(f"{row.iata}: {row.airport}"),
        icon=folium.Icon(color="blue", icon="plane")
    ).add_to(marker_cluster)

# add delays to layer control
fg1 = folium.FeatureGroup(name="Delays", control=True).add_to(m_airports)
HeatMap(delay_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg1)

# add cancellations to layer control
fg2 = folium.FeatureGroup(name="Cancellations", control=True).add_to(m_airports)
HeatMap(cancel_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg2)

# add layer control to the map
folium.LayerControl(collapsed=False).add_to(m_airports)
m_airports